In [75]:
import json
from tensorflow.keras import optimizers
import numpy as np

data = []
endata = []
chdata = []

def chine(string):
    for ch in string :
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

for line in open('testing.json', 'r', encoding='utf-8'):
    data.append(json.loads(line))  

for i in range(len(data)):
    if chine(data[i]['chinese']) == True :
        if len(data[i]['chinese']) < 10:
            endata.append(data[i]['english'])
            chdata.append('@'+data[i]['chinese']+'。')
        if len(chdata) == 100 :
            break

en_vocab = set(''.join(endata))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(chdata))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('down')




down


In [76]:
en_num_data = [[en2id[en] for en in line ] for line in endata]
ch_num_data = [[ch2id[ch] for ch in line] for line in chdata]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in chdata]

print('char:', endata[1])
print('index:', en_num_data[1])
print(len(en_num_data))
len(ch_num_data)
# de_num_data

char: Choose a recorder.
index: [56, 59, 29, 29, 13, 21, 18, 50, 18, 53, 21, 5, 29, 53, 12, 21, 53, 60]
100


100

In [74]:
import numpy as np

max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 49
max decoder length: 10
index data:
 [56, 59, 29, 29, 13, 21, 18, 50, 18, 53, 21, 5, 29, 53, 12, 21, 53, 60]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [77]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.01
BATCH_SIZE = 20
EPOCHS = 250


In [78]:
# ======================================keras model==================================
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np

# ==============encoder=============
encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
#emb_inp = Embedding(output_dim=HIDDEN_SIZE, input_dim=EN_VOCAB_SIZE)(encoder_inputs)
encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)

In [79]:
decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

In [80]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.)

Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_47 (InputLayer)           [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
input_48 (InputLayer)           [(None, None, 403)]  0                                            
__________________________________________________________________________________________________
lstm_20 (LSTM)                  [(None, None, 256),  330752      input_47[0][0]                   
__________________________________________________________________________________________________
lstm_22 (LSTM)                  [(None, None, 256),  675840      input_48[0][0]                   
                                                                 lstm_20[0][1]             

5/5 [==============================] - 1s 218ms/step - loss: 0.3874 - accuracy: 0.6670
Epoch 61/250
5/5 [==============================] - 1s 207ms/step - loss: 0.3788 - accuracy: 0.6620
Epoch 62/250
5/5 [==============================] - 1s 170ms/step - loss: 0.4035 - accuracy: 0.6570
Epoch 63/250
5/5 [==============================] - 1s 171ms/step - loss: 0.4247 - accuracy: 0.6620
Epoch 64/250
5/5 [==============================] - 1s 169ms/step - loss: 0.3902 - accuracy: 0.6650
Epoch 65/250
5/5 [==============================] - 1s 174ms/step - loss: 0.3886 - accuracy: 0.6650
Epoch 66/250
5/5 [==============================] - 1s 170ms/step - loss: 0.3665 - accuracy: 0.6630
Epoch 67/250
5/5 [==============================] - 1s 171ms/step - loss: 0.3687 - accuracy: 0.6630
Epoch 68/250
5/5 [==============================] - 1s 170ms/step - loss: 0.3698 - accuracy: 0.6590
Epoch 69/250
5/5 [==============================] - 1s 169ms/step - loss: 0.3556 - accuracy: 0.6590
Epoch 70/250


5/5 [==============================] - 1s 263ms/step - loss: 0.2566 - accuracy: 0.6890
Epoch 142/250
5/5 [==============================] - 1s 230ms/step - loss: 0.2370 - accuracy: 0.6860
Epoch 143/250
5/5 [==============================] - 1s 246ms/step - loss: 0.2320 - accuracy: 0.6880
Epoch 144/250
5/5 [==============================] - 1s 228ms/step - loss: 0.2209 - accuracy: 0.6870
Epoch 145/250
5/5 [==============================] - 1s 222ms/step - loss: 0.2297 - accuracy: 0.6900
Epoch 146/250
5/5 [==============================] - 1s 218ms/step - loss: 0.2359 - accuracy: 0.6870
Epoch 147/250
5/5 [==============================] - 1s 221ms/step - loss: 0.2207 - accuracy: 0.6920
Epoch 148/250
5/5 [==============================] - 1s 218ms/step - loss: 0.2211 - accuracy: 0.6850
Epoch 149/250
5/5 [==============================] - 1s 200ms/step - loss: 0.2243 - accuracy: 0.69000s - loss: 0.2236 - accuracy: 0.
Epoch 150/250
5/5 [==============================] - 1s 170ms/step - loss

5/5 [==============================] - 1s 171ms/step - loss: 0.1624 - accuracy: 0.7140
Epoch 222/250
5/5 [==============================] - 1s 173ms/step - loss: 0.1492 - accuracy: 0.7150
Epoch 223/250
5/5 [==============================] - 1s 171ms/step - loss: 0.1706 - accuracy: 0.7160
Epoch 224/250
5/5 [==============================] - 1s 169ms/step - loss: 0.2360 - accuracy: 0.6980
Epoch 225/250
5/5 [==============================] - 1s 171ms/step - loss: 0.2637 - accuracy: 0.6950
Epoch 226/250
5/5 [==============================] - 1s 178ms/step - loss: 0.2355 - accuracy: 0.6960
Epoch 227/250
5/5 [==============================] - 1s 172ms/step - loss: 0.2089 - accuracy: 0.7070
Epoch 228/250
5/5 [==============================] - 1s 184ms/step - loss: 0.1725 - accuracy: 0.7060
Epoch 229/250
5/5 [==============================] - 1s 172ms/step - loss: 0.1564 - accuracy: 0.7180
Epoch 230/250
5/5 [==============================] - 1s 169ms/step - loss: 0.1342 - accuracy: 0.7260
Epoc

In [81]:
encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])

# 預測模型中的decoder的初始化狀態需要傳入新的狀態
decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))

# 使用傳入的值來初始化當前模型的輸入狀態
decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])
decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
decoder_outputs = decoder_dense(decoder_h2)

decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], 
                      [decoder_outputs, state_h1, state_c1, state_h2, state_c2])

In [82]:
for k in range(0,100):
    test_data = encoder_input_data[k:k+1]
    h1, c1, h2, c2 = encoder_model.predict(test_data)
    target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
    target_seq[0, 0, ch2id['@']] = 1
    outputs = []
    while True:
        output_tokens, h1, c1, h2, c2 = decoder_model.predict([target_seq, h1, c1, h2, c2])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        outputs.append(sampled_token_index)
        target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
        target_seq[0, 0, sampled_token_index] = 1
        if sampled_token_index == ch2id['。']: break
        # if len(outputs) > 11: break
    
    print(endata[k])
    print(''.join([id2ch[i] for i in outputs]))


Look at these coasters over here.
一言为定。
Choose a recorder.
选择一个记录员.。
I hadn't paid the telephone bill.
我锁好了所有的门。
That's easier said than done, of course.
当然，知易行难。
Side-to-Side Movements.
象官员说，他们…。
about like 80 degrees.
没人看的见你摇头。
We all are from Shandong.
俺都是山东人。
She was possessed by a devil.
她着了魔。
This wool knits up well.
这种毛线很好织。
The majority was wrong last time.
凶犯当场被抓住。
Stone Soup Stories to Go!
今朝是三点过几分。
Done. See you tomorrow.
交易由管理员批准。
He eased some of the strains on the poor.
缓解了穷人的压力。
Could it be that it was written wrongly?
莫非写错了?。
What a terrible temper!
脾气真够坏的！。
Great talents flower late.
大器晚成。
I forbid you to make a sortie today.
你今天不许出击。
C：My surname is Jiang.
C：我姓姜。
Well, if it was greater .
如果拉力很大。
They looked over to the left.
她们朝左边看。
To supervise the management of printing industry.
监督管理印刷业。
no one else can see you shake your head.
没人看的见你摇头。
All photos dials.
所有照片刻盘。
Stained glass window panels;
她着了魔。
The murderer was caught red-handed.
凶犯当场被抓住。
You don’t love Melanie